In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from Points import Points
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import chain
import mediapipe as mp
import glob
import os
import cv2

/Users/Fritz/opt/anaconda3/envs/PyCharm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [101]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For static images:
photo_landmark_dict = {}
list_paths = []
shapes = []

# CK+ load
for emotion in ["anger", "contempt", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]:
    emo_folder_path = "../Datasets/CK+/" + emotion
    for pic_file in glob.glob(os.path.join(emo_folder_path, "*png")):
        list_paths.append([emotion, pic_file])

# JAFFE load
for pic_file in glob.glob(os.path.join("../Datasets/JAFFE", "*tiff")):
    if "AN" in pic_file[16:]:
        list_paths.append(["anger", pic_file])
    elif "DI" in pic_file[16:]:
        list_paths.append(["disgust", pic_file])
    elif "FE" in pic_file[16:]:
        list_paths.append(["fear", pic_file])
    elif "HA" in pic_file[16:]:
        list_paths.append(["happiness", pic_file])
    elif "NE" in pic_file[16:]:
        list_paths.append(["neutral", pic_file])
    elif "SA" in pic_file[16:]:
        list_paths.append(["sadness", pic_file])
    else:
        list_paths.append(["surprise", pic_file])

# FER-2013
FER_emo_dict = {0:"anger", 1:"disgust", 2:"fear", 3:"happiness", 4:"neutral", 5:"sadness", 6:"surprise"}
for set in ["test/", "train/"]:
    root = "../Datasets/FER2013/" + set
    for index, emotion in enumerate(["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]):
        emo_folder_path = root + emotion
        for pic_file in glob.glob(os.path.join(emo_folder_path, "*jpg")):
            list_paths.append([FER_emo_dict[index], pic_file])

In [100]:
angie = 0
disgust = 0
fe = 0
ha = 0
ne = 0
sa = 0
su = 0
for pic_file in glob.glob(os.path.join("../Datasets/JAFFE/", "*tiff")):
    if "AN" in pic_file[16:]:
        angie +=1
    elif "DI" in pic_file[16:]:
        disgust +=1
    elif "FE" in pic_file[16:]:
        fe +=1
    elif "HA" in pic_file[16:]:
        ha +=1
    elif "NE" in pic_file[16:]:
        ne +=1
    elif "SA" in pic_file[16:]:
        sa +=1
    else:
        su +=1
print(angie, disgust, fe, ha, ne, sa, su)

30 29 32 31 30 31 30


In [ ]:
skipped = 0

drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.2) as face_mesh:
    for idx, [emo, file] in enumerate(list_paths):
        image = cv2.imread(file, 0)
        # Convert the BGR image to RGB before processing.
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if results.multi_face_landmarks is not None:
            photo_landmark_dict[idx-skipped] = [emo]
            x_coords = []
            y_coords = []
            z_coords = []
            for count, mark in enumerate(results.multi_face_landmarks[0].landmark):
                x_coords.append(mark.x)
                y_coords.append(mark.y)
                z_coords.append(mark.z)

                # MUST CROP PHOTO FIRST AND THEN STUFF.
            shape = image.shape

            x_coords = [x * shape[0] for x in x_coords]
            y_coords = [y * shape[1] for y in y_coords]

            range_x = max(x_coords) - min(x_coords)
            range_y = max(y_coords) - min(y_coords)
            offset_x = [x - min(x_coords) for x in x_coords]
            offset_y = [y - min(y_coords) for y in y_coords]
            new_x = [x / range_x for x in offset_x]
            new_y = [y / range_y for y in offset_y]

            photo_landmark_dict[idx-skipped][1:479] = zip(new_x, new_y, z_coords)
        else:
            skipped += 1
        if idx % 200 == 0:
            print(idx)
            print(f"length: {len(photo_landmark_dict[idx-skipped])}")


print(skipped)
#print(len(photo_landmark_dict[0]))

In [103]:
'''for a in photo_landmark_dict.keys():
    if len(photo_landmark_dict[a]) != 479:
        print(a)
        print(f"weird thing: {photo_landmark_dict[a]}")'''
        
dataframe = pd.DataFrame(data=photo_landmark_dict)
#print(dataframe.head())
#print(dataframe.tail())
dataframe.to_json("./photo_landmark_list.json", orient='columns')

LINE BREAK :)

In [7]:
emotion_dict = {"anger": 0, "contempt": 1, "disgust": 2, "fear": 3, "happiness": 4, "neutral": 5, "sadness": 6,"sad": 6,
            "surprise": 6}

def read_data(path, label_points):
    '''Path for json with '''
    landmarks = pd.read_json(path)
    emotions = np.asarray([emotion_dict[y]/6 for y in landmarks.iloc[0].astype(object)], dtype=np.float32)
    columns = landmarks.shape[1]
    points = []
    for photo in range(columns):
        for point in label_points:
            coords = landmarks[photo].iloc[point+1][0:2]
            points.extend(coords)

    points = np.array(points, dtype=np.float32).reshape(35836, len(label_points*2)) # EDIT THINGS HERE

    print(f"shape at the end: {points.shape}")
    return emotions, points
    
def select_points(emotions, points, training_ratio):
    X_train, X_test, y_train, y_test = train_test_split(points, emotions, 
                                                        test_size=(1-training_ratio), random_state=1234)
    
    X_train = torch.from_numpy(X_train)
    X_test = torch.from_numpy(X_test)
        
    
    y_train = torch.from_numpy(y_train)
    y_test = torch.from_numpy(y_test)
    
    
    return X_train, X_test, y_train, y_test
    

In [8]:
# 1)
min_wanted_points = Points.right_eye_middle.value + Points.left_eye_middle.value + Points.nose.value + Points.mouth_inner.value
max_wanted_points = min_wanted_points + Points.right_eye_inner.value + Points.right_eye_outer.value + \
        Points.left_eye_inner.value + Points.left_eye_outer.value

emotions, points = read_data("./photo_landmark_list.json", max_wanted_points)
X_train, X_test, y_train, y_test = select_points(emotions, points, .95)

shape at the end: (35836, 268)


In [84]:
y_test.reshape(1792,1)

tensor([[6.],
        [4.],
        [0.],
        ...,
        [4.],
        [4.],
        [6.]])

In [37]:
# Neural Net model

# 1) Model
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(n_input_features, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 7)

    def forward(self, x):
        x = x.view(-1, 268)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Model(268)

In [9]:
# Neural Net model

# 1) Model
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(n_input_features, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 1)
        
        self.t = nn.Threshold(.5, 1)

    def forward(self, x):
        x = x.view(-1, 268)
        x = self.t(self.fc1(x))
        x = self.t(self.fc2(x))
        x = self.t(self.fc3(x))
        x = self.fc4(x)
        X = 1
        x = F.softmax(x, dim=X)
        return x

model = Model(268)

In [10]:
# 2) Loss and optimizer
num_epochs = 1
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
#criterion = nn.L1Loss(size_average=None, reduce=None, reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# 3) Training loop
counter = 0
for epoch in range(num_epochs):
    for index, row in enumerate(X_train):

        optimizer.zero_grad()

        # Forward pass and loss
        y_pred = model(row)
        print(y_pred)
        print(y_train[index])
        loss = criterion(y_pred, y_train[index].type(torch.LongTensor).view(1))

        # Backward pass and update
        loss.backward()
        optimizer.step()

        if (index+1) % 1000 == 0:
            print(f'row: {index+1}, loss = {loss.item():.8f}')



with torch.no_grad():
    correct = 0
    neutrals = 0
    total = 1792
    y_predicted = model(X_test)
    for row in range(y_predicted.shape[0]):
        print(row)
        break
        min_dist = 100
        closest_index = None
        for index in range(7):
            index_dist = abs(float(y_predicted[row][index])-1)
            if index_dist < min_dist:
                dist = index_dist
                closest_index = index

        if closest_index == y_test[row]:
            correct += 1
        elif 5 == y_test[row]:
            neutrals += 1
        else:
            #print(f"wrong row: {y_predicted[row]}")
            #print(f"Correct row: {y_test[row]} \n")
            continue
    print(f"correct: {correct}")
    print(f"neutrals: {neutrals}")
    print(f"total: {total}")

#torch.save(model.state_dict(), "./model-v10.pt")

tensor([[1.]], grad_fn=<SoftmaxBackward0>)
tensor(1.)


IndexError: Target 1 is out of bounds.

# 